In [224]:
import pandas as pd
import csv
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn import metrics

In [225]:
North_east_2 = pd.read_csv("North_east_2.csv")
North_east = pd.read_csv("North_east.csv")
North_west_2 = pd.read_csv("North_west_2.csv")
North_west_1 = pd.read_csv("North_west_1.csv")

In [226]:

dfs = [North_east_2, North_east,North_west_2,North_west_1]
df=pd.DataFrame()


for i in dfs:
    df = df.append(i)

data_mean=df.groupby('Datetime').mean()
data_mean.index = pd.to_datetime(data_mean.index)

In [227]:
data_mean.columns

Index(['air_temp_5cm_avg', 'air_temp_5cm_max', 'air_temp_5cm_min',
       'air_temp_avg', 'air_temp_max', 'air_temp_min', 'cloud_cover_avg',
       'cloud_cover_max', 'cloud_cover_min', 'dew_point_avg', 'dew_point_max',
       'dew_point_min', 'ice_acc_period', 'liquid_acc_period',
       'long_wave_radiation_avg', 'long_wave_radiation_max',
       'long_wave_radiation_min', 'pet_period', 'precip_acc_period',
       'precip_acc_period_adjusted', 'precip_acc_period_raw',
       'precip_duration', 'relative_humidity_avg', 'relative_humidity_max',
       'relative_humidity_min', 'short_wave_radiation_avg',
       'short_wave_radiation_max', 'short_wave_radiation_min',
       'snow_acc_period', 'sunshine_duration', 'visibility_avg',
       'visibility_max', 'visibility_min', 'wind_gust_max',
       'wind_speed_2m_avg', 'wind_speed_2m_max', 'wind_speed_2m_min',
       'wind_speed_avg', 'wind_speed_max', 'wind_speed_min'],
      dtype='object')

In [228]:
price =  pd.read_csv("London Wheat Futures Historical Data.csv")
price['Datetime'] = pd.to_datetime(price['Date'])
price = price.set_index(['Datetime'])
price = price.drop(['Date'], axis=1)

fdata = pd.concat([data_mean, price['Price']], axis=1)
fdata.to_csv('data.csv')



In [229]:
Price = price.groupby(pd.Grouper(freq='10D')).mean()


In [230]:

def rf_r2score(dataset,price):
    X = dataset.values
    y = price.iloc[:, price.columns.isin(['Price'])].values
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=100)

    rf_exp = RandomForestRegressor(n_estimators= 200, random_state=1000)
    rf_exp.fit(X_train, y_train)

    y_pred = rf_exp.predict(X_test)
    print("R2: ",r2_score(y_test,y_pred))
    print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
   


In [425]:
data_copy = fdata

data_copy['frost_days'] = fdata[fdata.air_temp_min<0 ]['air_temp_min']
data_copy['ice_days'] = fdata[fdata.air_temp_max<0 ]['air_temp_max']
data_copy['tem_10_25'] = fdata[(10<fdata.air_temp_avg)&(fdata.air_temp_avg<25) ]['air_temp_avg']
data_copy['wet_days'] = fdata[fdata.precip_acc_period>1 ]['precip_acc_period']
data_copy['very_heavy_precip_days'] = fdata[fdata.precip_acc_period>20 ]['precip_acc_period']
data_copy['heavy_precip_days'] = fdata[fdata.precip_acc_period>10 ]['precip_acc_period']
data_copy['Summer_days'] = fdata[fdata.air_temp_max>25 ]['air_temp_max']
data_copy['tropical_nights'] = fdata[fdata.air_temp_min>20 ]['air_temp_min']
data_copy['sunshine'] = fdata[(fdata.sunshine_duration>10) ]['sunshine_duration']

data_1 = data_copy.groupby(pd.Grouper(freq='10D')).agg({'frost_days':'count', 
                                               'ice_days':'count',
                                               'wind_gust_max':'sum',
                                               'heavy_precip_days':'count',
                                               'wet_days':'count',
                                                'cloud_cover_avg':'sum',
                                               'precip_acc_period':'sum', 
                                              'sunshine_duration':'sum',
                                        
                                            'relative_humidity_avg':['mean'],'relative_humidity_min':['min','mean'],
                                            'relative_humidity_max':['max','min','mean'],
                                             'wind_speed_max':'max','wind_speed_min':'max','wind_speed_avg':'max',
                                              
                                              'air_temp_max':['max','min','mean'],'air_temp_min':['min','mean'],
                                            
                                                         
                                            
                         })

winter =data_1.loc[(data_1.index.month.isin([  1, 2]))]
winter_price = Price.loc[(Price.index.month.isin([  1, 2]))]
print(winter.corrwith(winter_price['Price']))
print(rf_r2score(winter,winter_price))

frost_days             count   -0.034573
ice_days               count    0.021877
wind_gust_max          sum     -0.387864
heavy_precip_days      count   -0.074010
wet_days               count   -0.187004
cloud_cover_avg        sum      0.627717
precip_acc_period      sum     -0.164074
sunshine_duration      sum      0.158435
relative_humidity_avg  mean     0.037454
relative_humidity_min  min      0.094521
                       mean     0.125583
relative_humidity_max  max      0.012466
                       min     -0.143658
                       mean    -0.084310
wind_speed_max         max     -0.123619
wind_speed_min         max      0.091394
wind_speed_avg         max     -0.020655
air_temp_max           max      0.164925
                       min     -0.093587
                       mean     0.031363
air_temp_min           min      0.062020
                       mean     0.096499
dtype: float64


<ipython-input-230-0c5437bf5394>:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf_exp.fit(X_train, y_train)


R2:  0.5781491214643477
Root Mean Squared Error: 22.83757677239442
None


In [424]:
data_spring = data_copy.groupby(pd.Grouper(freq='10D')).agg({'frost_days':'count', 
                                               
                                               'tem_10_25':'count',
                                               'wind_gust_max':'sum',
                                               'heavy_precip_days':'count',
                                               'Summer_days':'count',
                                               'wet_days':'count',
                                                         
                                               'precip_acc_period':'sum',
                                                'cloud_cover_avg':'sum',
                                              'sunshine_duration':'sum',
                                              'wind_speed_max':['mean','max'],'wind_speed_min':['max'],'wind_speed_avg':['max'],
                                              'air_temp_max':['max','min','mean'],'air_temp_min':['max','min','mean'],
                                              'air_temp_avg':'mean'
                         })

spring =data_spring.loc[(data_spring.index.month.isin([3,4]))]
spring_price = Price.loc[(Price.index.month.isin([3,4]))]
print(spring.corrwith(spring_price['Price']))
print(rf_r2score(spring,spring_price))



frost_days         count    0.206482
tem_10_25          count    0.228103
wind_gust_max      sum     -0.365380
heavy_precip_days  count   -0.042375
Summer_days        count    0.100839
wet_days           count   -0.067332
precip_acc_period  sum     -0.054397
cloud_cover_avg    sum      0.494284
sunshine_duration  sum      0.305869
wind_speed_max     mean    -0.058406
                   max      0.008937
wind_speed_min     max      0.156225
wind_speed_avg     max      0.069153
air_temp_max       max      0.070431
                   min      0.053291
                   mean     0.076671
air_temp_min       max      0.173527
                   min      0.006188
                   mean     0.077547
air_temp_avg       mean     0.070952
dtype: float64


<ipython-input-230-0c5437bf5394>:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf_exp.fit(X_train, y_train)


R2:  0.7193280268724688
Root Mean Squared Error: 16.862392610557283
None


In [430]:
data_summer = data_copy.groupby(pd.Grouper(freq='10D')).agg({ 
                                               
                                               'tem_10_25':'count',
                                               'wind_gust_max':'sum',
                                               'heavy_precip_days':'count',
                                               'Summer_days':'count',
                                               'precip_acc_period':'sum',
                                              'sunshine_duration':'sum',
                                               'cloud_cover_avg':'sum',
                                            'relative_humidity_avg':['mean'],'relative_humidity_min':['min','mean'],
                                            'relative_humidity_max':['max','min'],
                                            'wind_speed_max':['mean','max'],'wind_speed_min':['max'],'wind_speed_avg':['mean'],
                    
                                              'air_temp_max':['max','min','mean'],'air_temp_min':['max','min','mean'],
                                              'air_temp_avg':'mean'
                         })



summer =data_summer.loc[(data_summer.index.month.isin([6,7]))]
summer_price = Price.loc[(Price.index.month.isin([6,7]))]
print(summer.corrwith(summer_price['Price']))
print(rf_r2score(summer,summer_price))




tem_10_25              count    0.192681
wind_gust_max          sum     -0.305467
heavy_precip_days      count    0.126702
Summer_days            count    0.110207
precip_acc_period      sum      0.213097
sunshine_duration      sum      0.346832
cloud_cover_avg        sum      0.372510
relative_humidity_avg  mean     0.072091
relative_humidity_min  min     -0.038879
                       mean     0.019020
relative_humidity_max  max      0.090858
                       min      0.111258
wind_speed_max         mean     0.133083
                       max      0.212340
wind_speed_min         max      0.028345
wind_speed_avg         mean     0.094975
air_temp_max           max     -0.001895
                       min     -0.033332
                       mean    -0.047414
air_temp_min           max     -0.113711
                       min     -0.059166
                       mean    -0.028206
air_temp_avg           mean    -0.054537
dtype: float64


<ipython-input-230-0c5437bf5394>:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf_exp.fit(X_train, y_train)


R2:  0.38963302252588716
Root Mean Squared Error: 16.757288571597925
None


In [416]:
corr = fdata.corr()
corr.style.background_gradient(cmap='coolwarm')

/Users/yihangu/opt/anaconda3/lib/python3.8/site-packages/pandas/io/formats/style.py:1126: RuntimeWarning: All-NaN slice encountered
  smin = np.nanmin(s.to_numpy()) if vmin is None else vmin
/Users/yihangu/opt/anaconda3/lib/python3.8/site-packages/pandas/io/formats/style.py:1127: RuntimeWarning: All-NaN slice encountered
  smax = np.nanmax(s.to_numpy()) if vmax is None else vmax
